<a href="https://colab.research.google.com/github/google/applied-machine-learning-intensive/blob/master/applied-machine-learning-intensive/amli/v2/content/03_regression/01_introtoscikit/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Copyright 2019 Google LLC.

In [0]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introduction to scikit-learn

<img height="20px" src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/05/Scikit_learn_logo_small.svg/220px-Scikit_learn_logo_small.svg.png" align="left" hspace="10px"> [Scikit-learn](https://scikit-learn.org) is a machine learning library for Python.

We will explore multiple machine learning libraries throughout this course. Scikit-learn was chosen to be the first due to the approachability of the library and its support for a wide variety of traditional machine learning models, not just deep learning models.

## Overview

### Learning Objectives

* Understand the scikit-learn programming model
* Navigate the variety of features that scikit-learn provides
* Using scikit-learn demonstrate the ability to:
  * load sample data
  * generate sample data
  * transform data
  * train a model
  * make predictions using a model
  * create a data processing and model training pipeline
  * create metrics about model performance
  * visualize predictions returned from a model

### Prerequisites

* Introduction to Colab
* Intermediate Python
* Introduction to Pandas
* Visualizations

### Estimated Duration

60 minutes

### Grading Criteria

Each exercise is worth 3 points. The rubric for calculating those points is:

| Points | Description |
|--------|-------------|
| 0      | No attempt at exercise |
| 1      | Attempted exercise, but code does not run |
| 2      | Attempted exercise, code runs, but produces incorrect answer |
| 3      | Exercise completed successfully |

There are 4 exercises in this Colab so there are 12 points available. The grading scale will be 9 points.

## Datasets

Scikit-learn contains methods for loading, fetching, and making (generating) data. The methods for doing this all fall under the [datasets](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.datasets) package. Most of the functions in this package have `load`, `fetch`, or `make` in the name to let you know what the method is doing under the hood.

**Loading** functions bring static datasets into your program. The data comes pre-packaged with scikit-learn, so no network access is required.

**Fetching** functions also bring static datasets into your program. However, the data is pulled from the internet (and cached), so if you don't have network access these functions might fail.

**Generating** functions make dynamic datasets based on some equation.

These pre-packaged dataset functions exist for many popular/classic datasets such as the [MNIST digits dataset](https://en.wikipedia.org/wiki/MNIST_database) and the [Iris flower dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set). The generation functions reference classic dataset "shape" formations such as [moons](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_moons.html#sklearn.datasets.make_moons) and [swiss rolls](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_swiss_roll.html#sklearn.datasets.make_swiss_roll).

These datasets are great for getting introduced to machine learning.

### Loading

Let us first look at an example of loading data. We will load the iris flowers dataset using the [load_iris](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html#sklearn.datasets.load_iris) function.

In [0]:
from sklearn.datasets import load_iris

iris_data = load_iris()
iris_data

That's a lot to take in!

Let's examine this loaded data a little more closely. First we'll see what type the data is:

In [0]:
type(iris_data)

A `sklearn.utils.Bunch` is a class type that you'll see quite often when working with datasets built into scikit-learn. It is a dictionary-like container for feature and target data within a dataset.

You won't find much documentation about `Bunch` objects though because they are not really meant for usage beyond containing data returned by scikit-learn.

Let's look at the attributes of our iris data bunch:

In [0]:
dir(iris_data)

`DESCR` is a description of the dataset.

In [0]:
print(iris_data['DESCR'])

`filename` is the file where the data is stored.

In [0]:
print(iris_data['filename'])

`feature_names` are the names of the feature columns.

In [0]:
print(iris_data['feature_names'])

`target_names` are not however the names of the target columns. There is only one column of targets.

Instead, `target_names` are the human-readable names of the classes in the target list within the bunch. In this case they are the names of the three species of iris in this dataset.

Note that the target names are in a list where:

  * setosa is the 0th element
  * versicolor is the 1st element
  * virginica is the 2nd element

In [0]:
print(iris_data['target_names'])

We can now examine `target` and see that it contains zeros, ones, and twos. These correspond to the target names 'setosa', 'versicolor', and 'virginica'.

In [0]:
print(iris_data['target'])

Last we look at the `data` within the bunch. The data is an array of arrays. Each sub-array contains four values. These values match up with the `feature_names`. The first item in each sub-array is 'sepal length (cm)', the next is 'sepal width (cm)', and so on.

In [0]:
iris_data['data']

The number of target values should always equal the number of rows in the data.

In [0]:
print(len(iris_data['data']))
print(len(iris_data['target']))

`Bunch` objects are a perfectly fine container for data. They can be used directly to feed models.

`Bunch` objects are *not* very good for analyzing and manipulating your data.

In this course we will typically convert `Bunch` objects into Pandas `DataFrame` objects to make analysis, data cleaning, and test/train splitting easier and more uniform.

To do this we will take the matrix of feature data and append the target data to it to create a single matrix of data. We also take the list of feature names and append the word 'species' to represent the target classes in the matrix.

An example of how to do this is below.

In [0]:
import pandas as pd
import numpy as np

iris_df = pd.DataFrame(
  data=np.append(
    iris_data['data'], 
    np.array(iris_data['target']).reshape(len(iris_data['target']), 1), 
    axis=1),
  columns=np.append(iris_data['feature_names'], ['species'])
)

iris_df.sample(n=10)

You might notice that the integer representation of species got converted to a floating point number along the way. We can change that back.

In [0]:
iris_df['species'] = iris_df['species'].astype('int64')

iris_df.sample(n=10)

### Fetching

Fetching is similar to loading. Scikit-learn will first see if it can find the dataset locally and if so will simply load the data. Otherwise, it will attempt to pull the data from the internet.

We can see fetching in action with the [fetch_california_housing](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html#sklearn.datasets.fetch_california_housing) function below. If you run the code block once you should see a message that the data is downloading. If you run it again you won't see that message because the data is already local to your running code.

In [0]:
from sklearn.datasets import fetch_california_housing

housing_data = fetch_california_housing()

type(housing_data)

The dataset is once again given to us as a `Bunch`.

If you followed the link to the [fetch_california_housing](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html#sklearn.datasets.fetch_california_housing) you notice that the dataset is a **regression** dataset as opposed the iris dataset, which was a **classification** dataset.

We can see the difference in the dataset by checking out the attributes of the `Bunch`.

In [0]:
dir(housing_data)

We see that four of the attributes that we expect are present, but 'target_names' is missing. This is because our target is now a home price and not a discrete value like an iris species.

In [0]:
print(housing_data['target'][:10])

Converting a `Bunch` of regression data to a `DataFrame` is no different than converting a `Bunch` of classification data.

In [0]:
import pandas as pd
import numpy as np

housing_df = pd.DataFrame(
  data=np.append(
    housing_data['data'], 
    np.array(housing_data['target']).reshape(len(housing_data['target']), 1), 
    axis=1),
  columns=np.append(housing_data['feature_names'], ['price'])
)

housing_df.sample(n=10)

### Generating

In the example datasets we've seen so far in this colab, the data is static and loaded from a file. Sometimes it makes more sense to generate a dataset. For this we use one of the many [generator](https://scikit-learn.org/stable/modules/classes.html#samples-generator) functions.

`make_regression` is a generator that will create a dataset with an underlying regression that you can then attempt to discover using various machine learning models.

In the example below we create a dataset with 10 data points. For the sake of visualization we have only one feature per datapoint, but we could ask for more.

The return value are the X and y values for the regression. The X is a matrix of features. The y is a list of targets.

Since a generator uses randomness to generate data, we are going to set a random_state in this colab for reproducibility. **You won't do this in your production code.**

In [0]:
from sklearn.datasets import make_regression

features, targets = make_regression(n_samples=10, n_features=1, random_state=42)

features, targets

We can use a visualization library to plot the regression data.

In [0]:
import matplotlib.pyplot as plt

plt.plot(features, targets, 'b.')
plt.show()

That data does have a very linear pattern!

If we want to make it more realistic, just add some noise during data generation.

**Remember that random_state is for reproducibility only. Don't use this in your code unless you have a good reason to!**

In [0]:
from sklearn.datasets import make_regression

features, targets = make_regression(n_samples=10, n_features=1, random_state=42, noise=5.0)

plt.plot(features, targets, 'b.')
plt.show()

There are dozens of dataset loaders and generators in the scikit-learn [datasets](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.datasets) package. When you want to play with a new machine learning algorithm, they are a great source of data for getting started.

## Models



Machine learning involves training a model to gain insight and predictive power from a dataset. Scikit-learn has support for many different types of models ranging from classic algebraic models through more modern deep learning models.

Throughout the remainder of this course you will learn about many of these models in much more depth. This section exists to survey some concepts that you will encounter when building and running models in scikit-learn.

### Estimators

Most of the models in scikit-learn are considered [estimators](https://scikit-learn.org/stable/modules/generated/sklearn.base.BaseEstimator.html#sklearn.base.BaseEstimator). An estimator is expected to implement two methods: `fit` and `predict`.

`fit` is used to train the model. At a minimum it is passed the feature data used to train the model. In supervised models it is also passed the target data.

`predict` is used to get predictions from the model. This method is passed features and returns target predictions.

Let's see an example of this in action.

A linear regression is a simple model that you might have encountered in a statistics class in the past. The model attempts to draw a straight line through a set of data points so that the line is as close to as many points as possible.

We'll use scikit-learn's [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) class to fit a line to the regression data that we generated earlier in this colab. To do that we simply call `fit(features, targets)`.

After fitting, we can ask the model for predictions. In this case we just ask for predictions based on the features that we used to train in order to draw a scatter plot of the actual data with the regression line plotted over it by calling `predict(features)`.

In [0]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression

regression = LinearRegression()
regression.fit(features, targets)
predictions = regression.predict(features)

plt.plot(features, targets, 'b.')
plt.plot(features, predictions, 'r-')
plt.show()

At this point, don't worry too much about the details of what `LinearRegression` is doing. There is a deep-dive into regression problems coming up soon.

For now just note the `fit`/`predict` pattern for training estimators and know that you'll see it throughout our adventures with scikit-learn.

### Transformers

In practice it is rare that you will get perfectly clean data that is ready to feed into your model for training (calling `fit`). Most of the time you will need to perform some type of cleaning on the data first.

You've got some hands-on experience doing this in our Pandas colabs. Scikit-learn can also be used to perform some data preprocessing tasks on your datasets.

Transformers are spread about within the scikit-learn library. Some are in the [preprocessing](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing) module while others are in more specialized packages like [compose](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.compose), [feature_extraction](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_extraction), [impute](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.impute), and more.

All transformers implement a `fit` and `transform` methods. The `fit` method calculates parameters necessary to perform the data transformation. `transform` actually applies the transformation. There is a convenience `fit_transform` method that performs both fitting and transformation in one method call.

Let's see a transformer in action.

We will use the [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler) to scale our feature data between zero and one.

Looking at our feature data now we can see values below zero and above one.

In [0]:
features

We will now create a `MinMaxScaler` and fit it to our feature data.

Each transformer has different information that it needs to perform a transformation. In the case of the `MinMaxScaler` the smallest and largest values in the data are needed.

In [0]:
from sklearn.preprocessing import MinMaxScaler

transformer = MinMaxScaler()
transformer.fit(features)
transformer.data_min_, transformer.data_max_

You might notice that the values are stored in arrays. This is because transformers can operate on more than one feature. In this case we have only one.

Next we need to apply the transformation to our features.

We can now see that all of the features fall between the range of zero to one.

In [0]:
features = transformer.transform(features)
features

### Pipelines

It isn't coincidence that transformers have `fit` and `transform` methods and that models have `fit` methods. The common interface across classes allows scikit-learn to create pipelines for data processing and model building.

A [pipeline](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.pipeline) is simply a series of transformers, often with an estimator at the end.

In the example below we use a [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) class to perform min-max scaling or our feature data and then train a linear regression model using the scaled features.

In [0]:
from sklearn.pipeline import Pipeline

features, targets = make_regression(n_samples=10, n_features=1, random_state=42, noise=5.0)

pipeline = Pipeline([
  ('scale', MinMaxScaler()),
  ('regression', LinearRegression())
])

pipeline.fit(features, targets)

predictions = pipeline.predict(features)

plt.plot(features, targets, 'b.')
plt.plot(features, predictions, 'r-')
plt.show()

### Metrics

So far we have seen ways that scikit-learn can help you get data to perform machine learning, modify that data, train a model, and finally make predictions. But how good are the predictions?

Scikit-learn also comes with many functions for measuring model performance in the [metrics](https://scikit-learn.org/stable/modules/classes.html#sklearn-metrics-metrics) package.

Later in this course you will learn about different ways to measure the performance of regression and classification models, as well as, the trade-offs between different metrics.

To illustrate a metrics function in action we'll import the [mean_squared_error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) function and use it to find the mean squared error (MSE) between the target values that we used to train our linear regression model and the predicted values.

In [0]:
from sklearn.metrics import mean_squared_error

mean_squared_error(targets, predictions)

What does the resulting value mean in relation to our model? Is it good or bad?

In this case it doesn't have much meaning aside from being the mean of the squares of the distance between our actual target values and their predicted values. Since the data that we fit the regression to isn't related to any real-world metrics the MSE is just a number.

As we learn more about machine learning and begin training models on real data you'll learn how to interpret MSE and other metrics in context of the data being analyzed and the problem being solved.

There are also metrics in each estimator class. These metrics can be extracted using the `score` method.

The `regression` class we created earlier can be scored, as can the `pipeline`.

In [0]:
print(regression.score(features, targets))
print(pipeline.score(features, targets))

The return value of the `score` method depends on the estimator being used. In the case of `LinearRegression` the score is the r-squared score, where scores closer to 1.0 are better. You can find the metric that `score` returns in the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression.score) for the estimator that you are using.

## Closing

Scikit-learn is a massive library that contains scores of resources for performing machine learning tasks. In this colab we have only introduced some basic concepts that you will see repeated throughout your career in data science.

We will cover other parts of scikit-learn and go into more depth later in this course.

You are also encouraged to check out the [scikit-learn documentation](https://scikit-learn.org/stable/documentation.html) where you will find a user's guide, tutorials, and a full API reference.

Scikit-learn is an Open Source project. You can find it on [Github](https://github.com/scikit-learn/scikit-learn)

## Resources

* https://scikit-learn.org/stable/documentation.html
* https://en.wikipedia.org/wiki/Scikit-learn
* https://en.wikipedia.org/wiki/Estimator
* https://en.wikipedia.org/wiki/Mean_squared_error
* https://github.com/scikit-learn/scikit-learn

# Exercises

## Exercise 1

Load the [Boston house price dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html) into a Pandas `DataFrame`. Append the target values to the last column of the `DataFrame` called `boston_df`. Name the target column 'PRICE'.

### Student Solution

In [0]:
# Your answer goes here

### Answer Key

**Solution**

In [0]:
from sklearn.datasets import load_boston

import pandas as pd
import numpy as np

boston_data = load_boston()

boston_df = pd.DataFrame(
  data=np.append(
    boston_data['data'], 
    np.array(boston_data['target']).reshape(len(boston_data['target']), 1), 
    axis=1),
  columns=np.append(boston_data['feature_names'], ['PRICE'])
)

**Validation**

In [0]:
import numpy as np

assert np.array_equal(boston_df.columns.values, [
  'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 
  'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'PRICE'
]), "Invalid columns"

assert int(boston_df.count().mean()) == 506, "Invalid row count"

assert \
  np.allclose(boston_df.max().values,
    [88.9762, 100., 27.74, 1., 0.871 , 8.78  ,100., 12.1265, 24., 711., 22., 
    396.9, 37.97, 50.]) and \
  np.allclose(boston_df.min().values,
    [6.3200e-03, 0.0000e+00, 4.6000e-01, 0.0000e+00, 3.8500e-01, 3.5610e+00,
    2.9000e+00, 1.1296e+00, 1.0000e+00, 1.8700e+02, 1.2600e+01, 3.2000e-01, 
    1.7300e+00, 5.0000e+00]) and \
  np.allclose(boston_df.mean().values,
    [3.61352356e+00, 1.13636364e+01, 1.11367787e+01, 6.91699605e-02,
       5.54695059e-01, 6.28463439e+00, 6.85749012e+01, 3.79504269e+00,
       9.54940711e+00, 4.08237154e+02, 1.84555336e+01, 3.56674032e+02,
       1.26530632e+01, 2.25328063e+01]), "Values look wrong"

"LGTM"

## Exercise 2

Search the [scikit-learn datasets documentation](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.datasets) and find a function to make a "Moons" dataset. Create a dataset with 75 samples. Use a random state of `42` and a noise of 0.08. Store the X return value in a variable called `features` and the y return value in a variable called `targets`.

### Student Solution

In [0]:
# Your answer goes here

### Answer Key

**Solution**

In [0]:
from sklearn.datasets import make_moons

features, targets = make_moons(n_samples=75, random_state=42, noise=0.08)

**Validation**

In [0]:
import numpy as np

expected_features = np.array([
  [ 0.97754808,  0.39475539],
  [ 1.78189288, -0.21831832],
  [ 0.65050093,  0.79957261],
  [ 0.92375778, -0.08382965],
  [-1.14624884, -0.02218683],
  [ 1.50110815, -0.46546351],
  [-0.72796104,  0.74708051],
  [ 0.51692696,  0.91378898],
  [ 1.83952168,  0.07913973],
  [ 1.77819003, -0.34058417],
  [-0.90880732,  0.3117695 ],
  [ 0.12540232, -0.04664424],
  [ 0.50184033, -0.35801865],
  [ 0.03742358,  1.05888979],
  [ 0.64473063,  0.63959507],
  [-0.35406285,  0.86244801],
  [ 0.91834691,  0.41354456],
  [-0.75524357,  0.17681177],
  [ 0.97987738, -0.46518352],
  [ 1.84785216,  0.05475808],
  [ 0.14421773,  0.39689783],
  [ 0.05571179,  0.31975237],
  [ 0.12482242,  0.95951448],
  [ 0.05289641, -0.07828259],
  [ 0.29897555, -0.14499539],
  [-0.92794884,  0.47121775],
  [ 0.8770787 ,  0.55311527],
  [ 1.27867297, -0.58246118],
  [ 0.26970088, -0.29120376],
  [-1.01110384,  0.65811134],
  [ 0.70161947, -0.40436449],
  [ 0.49651141, -0.43402886],
  [-0.06304992,  0.87951334],
  [ 1.47128822, -0.36324696],
  [-0.6412578 ,  0.75335289],
  [ 2.08693816,  0.58039895],
  [ 0.38727698,  0.89957458],
  [ 0.97194347, -0.60601875],
  [ 0.99296764,  0.41678474],
  [ 0.24435852,  0.8410018 ],
  [-0.05740765,  0.42868683],
  [ 0.66029429,  0.58932787],
  [ 1.97463263,  0.4431049 ],
  [ 0.86239319,  0.51735494],
  [-0.9590061 ,  0.0434791 ],
  [ 1.87526207, -0.08346191],
  [ 1.02073262, -0.46457981],
  [ 0.19030372, -0.24652299],
  [ 2.06739842,  0.46363273],
  [ 0.20119521,  0.94433112],
  [-0.71858376,  0.75646708],
  [ 0.15703781,  0.17211214],
  [-0.549251  ,  0.8359098 ],
  [ 0.42181883, -0.4194758 ],
  [-0.58221687,  0.92532465],
  [-0.00985648,  0.02182334],
  [ 1.9085485 , -0.04778112],
  [ 1.07890604, -0.55962958],
  [ 0.63886865,  0.85688863],
  [-0.96253176,  0.42724354],
  [ 2.01119727,  0.30994771],
  [ 1.3381743 , -0.46932677],
  [ 1.71051682, -0.34670448],
  [ 0.01205272,  0.4987428 ],
  [-0.92555797,  0.57155343],
  [ 1.10364816,  0.02839193],
  [ 0.63038566, -0.36777248],
  [-0.20807982,  1.21081344],
  [ 0.87932214,  0.23127916],
  [-0.39734699,  0.91105029],
  [-0.1008072 ,  0.98242972],
  [ 1.21794638, -0.49567763],
  [ 2.07688762,  0.34178425],
  [ 0.31217061,  0.97998039],
  [ 0.84985857, -0.43038397]])

assert np.allclose(features, expected_features), "Unexpected features"

expected_targets = np.array([
  0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0,
  0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1,
  0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1])

assert np.array_equal(targets, expected_targets), "Unexpected targets"

'LGTM'

## Exercise 3

In Exercise Two you created a "moons" dataset. In that dataset the features are (x,y)-coordinates that can be graphed in a scatterplot. The targets are zeros and ones that represent a binary classification.

Use matplotlib's [scatter](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.scatter.html) method to visualize the data as a scatterplot. Use the `c` argument to scatter to make the dots for each class a different color.

### Student Solution

In [0]:
# Your answer goes here

### Answer Key

**Solution**

In [0]:
from sklearn.datasets import make_moons

import matplotlib.pyplot as plt

features, targets = make_moons(n_samples=75, random_state=42, noise=0.08)

plt.scatter(features[:, 0], features[:, 1], c=targets)
plt.show()

**Validation**

In [0]:
# TODO

## Exercise 4

Use the [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) class to combine a data pre-processor and an estimator.

To accomplish this:

1. Find a [preprocessor](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing) that uses the max absolute value for scaling.
1. Find a [linear_model](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model) based on the Huber algorithm.
1. Combine this preprocessor and estimator into a pipeline.
1. Make a sample regression dataset with 200 samples, 1 feature. Use a random state of 85 and a noise of 5.0. Save the features in a variable called `features` and the targets in a variable called `targets`.
1. Fit the model.
1. Using the features that were created when the regression dataset was created, make predictions with the model and save them into a variable called `predictions`.
1. Plot the features and targets used to train the model on a scatterplot with blue dots.
1. Plot the features and predictions over the scatterplot as a red line.

### Student Solution

In [0]:
# Your answer goes here

### Answer Key

**Solution**

In [0]:
# Solution

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import HuberRegressor

pipeline = Pipeline([
  ('scale', MaxAbsScaler()),
  ('regression', HuberRegressor())
])

features, targets = make_regression(n_samples=200, n_features=1, random_state=85, noise=5.0)

pipeline.fit(features, targets)

predictions = pipeline.predict(features)

plt.plot(features, targets, 'b.')
plt.plot(features, predictions, 'r-')
plt.show()

**Validation**

In [0]:
# Validation

expected_features = np.array([
  [-1.37320909e+00],
  [ 4.13294171e-02],
  [-9.99948669e-01],
  [-9.95338151e-01],
  [-8.13332470e-01],
  [ 1.76850721e-01],
  [-1.32313256e+00],
  [ 2.45170471e-02],
  [ 5.91200499e-01],
  [-5.50947561e-01],
  [-1.03398074e+00],
  [ 1.81547474e+00],
  [ 2.37933102e-01],
  [ 1.38825013e+00],
  [ 1.01405789e+00],
  [-1.26917351e+00],
  [ 9.84757828e-01],
  [ 1.33395677e-01],
  [-3.44092397e-01],
  [-9.83887577e-02],
  [ 6.88071247e-01],
  [ 6.28582628e-01],
  [ 3.21251175e-01],
  [-1.26839208e+00],
  [-4.72347001e-01],
  [-1.10468058e+00],
  [-5.80300179e-01],
  [ 1.71458191e+00],
  [-8.78108389e-01],
  [ 1.10712328e+00],
  [-2.96874746e-01],
  [ 3.04793211e-01],
  [ 5.39050765e-01],
  [ 4.38242468e-01],
  [-1.53021166e-03],
  [-2.28902059e+00],
  [-3.45949491e-01],
  [ 1.73104067e+00],
  [ 4.11840282e-01],
  [ 1.03003721e+00],
  [-1.12090937e+00],
  [-4.00924524e-01],
  [ 2.81642191e-01],
  [ 9.30027197e-01],
  [ 8.20353999e-01],
  [-5.67303871e-01],
  [ 9.42197422e-01],
  [-3.43053946e-01],
  [-1.59562468e+00],
  [ 1.80189921e-01],
  [-4.60591592e-01],
  [ 8.13309578e-01],
  [-1.32358559e+00],
  [-1.42704812e+00],
  [-4.67476194e-03],
  [ 8.48404803e-01],
  [-1.04512273e+00],
  [ 5.86910888e-01],
  [-1.10602703e+00],
  [-1.05893758e+00],
  [ 5.93405161e-01],
  [-4.87382113e-01],
  [-4.04185104e-01],
  [ 1.38317737e+00],
  [ 1.03057868e+00],
  [-7.20612498e-02],
  [ 1.00195187e-01],
  [ 1.65746654e+00],
  [ 1.63555529e+00],
  [-1.76344400e+00],
  [-1.39928556e+00],
  [-5.23717798e-01],
  [-8.97540203e-02],
  [-1.07456699e+00],
  [ 7.93586305e-01],
  [-5.22438076e-01],
  [ 2.37775412e+00],
  [ 1.08685389e+00],
  [ 1.61166821e+00],
  [-7.72500851e-01],
  [-7.12057405e-01],
  [-4.07238748e-01],
  [-6.42177023e-01],
  [ 7.95017483e-01],
  [-3.81671601e-01],
  [ 3.89367783e-02],
  [ 1.13881760e-01],
  [-1.76475402e+00],
  [ 1.29529932e+00],
  [-8.34805672e-01],
  [-5.82507104e-01],
  [-6.29790059e-01],
  [ 9.96082287e-01],
  [ 6.61002918e-01],
  [ 2.69965540e-01],
  [ 1.46207893e+00],
  [-2.12789331e+00],
  [-1.11137129e+00],
  [ 8.83741119e-01],
  [-1.28071705e+00],
  [ 8.45780666e-01],
  [-2.08296632e-01],
  [-1.46175755e+00],
  [ 1.49095478e+00],
  [ 1.37619473e+00],
  [-1.49408672e+00],
  [-2.20501638e-01],
  [-4.98577067e-01],
  [-2.31932288e-01],
  [ 2.86922902e-01],
  [ 9.79870597e-01],
  [-6.31397967e-01],
  [ 1.55077235e+00],
  [ 2.61263847e+00],
  [ 2.08632906e+00],
  [-1.35444956e+00],
  [-1.02033134e+00],
  [ 6.47022662e-01],
  [ 9.75648164e-02],
  [-2.47605054e-01],
  [ 8.01853706e-01],
  [ 1.88665906e+00],
  [ 2.03581183e+00],
  [-6.72955148e-01],
  [-8.57704483e-01],
  [-7.28622758e-01],
  [-9.99525081e-01],
  [ 1.24279092e+00],
  [-8.56721061e-01],
  [-6.68866346e-01],
  [-3.97094747e-01],
  [-7.02473271e-01],
  [-7.64906504e-01],
  [-2.48759526e+00],
  [-7.54266445e-01],
  [-1.73426123e-01],
  [-6.27210581e-01],
  [-4.37931220e-02],
  [ 4.79324490e-01],
  [ 1.05348971e+00],
  [-1.27759849e+00],
  [-7.92648386e-01],
  [-5.77137518e-01],
  [-1.44312587e+00],
  [-5.79518698e-01],
  [-4.05259988e-01],
  [ 1.75195297e+00],
  [-8.47567151e-01],
  [ 1.94046613e-01],
  [ 2.61812827e-01],
  [-5.47422914e-01],
  [-1.91659827e+00],
  [ 9.07243402e-01],
  [-8.25518591e-01],
  [ 1.73049252e+00],
  [ 6.49705341e-01],
  [-1.31015080e+00],
  [-2.64003395e-01],
  [-1.45975318e+00],
  [-3.27094173e-01],
  [-6.71295517e-02],
  [ 1.19678471e+00],
  [ 7.29006161e-02],
  [ 9.98732857e-01],
  [-1.57780624e+00],
  [-1.28899836e+00],
  [ 1.05710645e+00],
  [-8.52728332e-01],
  [-1.25873007e+00],
  [-2.42304751e-01],
  [ 1.11943143e+00],
  [ 1.64589090e-01],
  [ 1.19218191e-01],
  [-1.54518158e+00],
  [-2.01068394e-01],
  [ 1.21247272e+00],
  [-7.83742897e-01],
  [-1.41250679e+00],
  [-1.28411633e+00],
  [-3.54861737e-01],
  [ 4.91432356e-01],
  [-1.00360614e+00],
  [-9.53866799e-01],
  [ 2.88950098e-01],
  [-8.80819768e-01],
  [-4.97490883e-01],
  [ 1.77991031e+00],
  [-3.53688640e-01],
  [ 7.55502609e-01],
  [-1.00955693e+00],
  [ 7.38235155e-01],
  [ 2.00279541e+00],
  [ 7.19081631e-02],
  [ 1.74217224e-01],
  [ 1.77287145e-01],
  [-1.61703227e+00],
  [ 3.75547664e-01],
  [-1.13014095e+00],
  [-4.40076564e-01],
  [-5.86043621e-01]])

assert np.allclose(features, expected_features), "Unexpected features"

expected_targets = np.array([
  -6.43930807e+00, -6.09693991e-01,  3.61257684e-01, -1.13671460e+00,
  -4.72507826e-01,  4.62303421e+00, -1.16058455e+01,  7.49123270e+00,
  5.47382619e+00,  1.33657206e+00, -1.06901876e+01,  4.01105349e+00,
  1.00882548e+00,  7.14911325e+00,  1.56482718e+01, -2.84264842e-01,
  3.92982148e+00,  1.00477211e+01, -1.54830345e+00, -7.49219920e+00,
  2.41175814e+00, -2.36353525e-01,  1.00264552e+01, -1.72083330e+01,
  -5.94791618e+00, -4.48616068e+00, -6.48184992e+00,  7.11006738e+00,
  -2.52312550e+00,  5.18048748e+00,  1.42756588e-01,  2.30205916e+00,
  5.27687543e+00,  6.28621083e+00,  1.83795796e+00, -1.40694487e+01,
  -5.87256361e+00,  1.74164101e+01, -1.85191747e+00,  1.64101070e+01,
  -5.29273521e+00, -4.64923366e+00, -5.15900171e+00, -6.55204146e-01,
  -6.01943039e+00, -1.32730559e+01,  8.29117076e+00, -6.35343374e+00,
  3.44927711e-02, -4.09680339e+00, -9.23788724e+00, -1.02075084e+00,
  -1.10575258e+01, -1.24529205e-01, -4.66648713e+00,  6.07584582e+00,
  -7.86311563e+00,  1.46409270e+01, -6.47879382e+00, -5.47392871e+00,
  1.49530219e-01,  1.54788145e+00,  1.29311901e+00,  1.50286732e+01,
  2.38349792e+00,  1.69393376e+00,  7.28169949e+00,  1.44513476e+01,
  1.83587982e+01, -1.15639868e+01, -1.62877063e+01, -5.75217605e+00,
  5.61120249e+00, -1.44635749e+01, -4.54115545e+00, -4.51570779e+00,
  3.20156912e+00,  2.72990003e+00, -5.15237919e+00,  1.01688507e+00,
  1.66118191e-01,  2.33628831e+00,  1.59477886e+00, -4.16298115e+00,
  4.47409174e+00, -3.63674639e+00,  8.12385866e+00, -5.73050199e+00,
  1.27035886e+01, -1.89117134e+00,  2.05474238e+00,  1.32261928e+01,
  3.96518500e+00,  1.41089903e+01, -2.10675135e+00,  1.50467160e+00,
  -9.90987786e+00, -1.29351396e+01,  8.11009190e+00, -3.06084090e+00,
  -9.00179765e-04, -9.17678742e-01, -9.67003358e+00,  7.53073836e+00,
  1.35891681e+01, -4.20536916e+00,  8.60296141e-02,  8.35563659e+00,
  -3.89423012e+00,  3.59972428e+00,  4.78325051e+00, -1.77013838e+00,
  3.47084674e+00,  9.01355091e+00,  1.57096868e+00, -3.84844215e+00,
  -6.93561083e+00,  1.46616354e+01,  9.27353385e-01,  1.68502311e+00,
  -1.69867355e+00,  5.22905356e+00,  1.57330064e+01, -3.89112086e+00,
  -7.57601210e+00,  3.06939596e-01,  2.94793855e+00,  2.61638296e+00,
  -5.28111231e+00,  2.38289808e+00, -4.38871541e+00, -3.77357770e+00,
  -2.07679376e-01, -1.84261181e+01,  5.75509708e+00,  7.27025431e+00,
  1.22945182e+00, -2.34948482e+00,  7.20288829e+00,  9.41999209e+00,
  -1.13493184e+01, -1.93656201e+00, -6.95906914e+00, -1.04626745e+01,
  -5.58303918e+00, -1.14103374e+01,  1.14048676e+01, -1.39716018e+01,
  -3.36619379e+00,  6.53189505e+00, -1.04170603e+01, -8.51915006e+00,
  5.29274079e+00,  2.45761731e+00,  8.58493974e+00,  8.40730398e+00,
  -8.36174810e+00, -2.04365527e+00, -7.60268615e+00, -5.81048794e-01,
  -1.62093095e+00,  9.21187416e+00, -2.46254550e+00,  1.34209425e+01,
  -6.82649529e+00, -1.26115726e+00,  4.15302888e+00, -1.84713845e+01,
  -3.07479976e+00,  1.46744739e+00,  7.13775775e+00, -8.07686274e+00,
  -8.19619976e+00, -1.20348785e+01, -3.46405895e+00,  3.73789115e+00,
  -6.50469936e-01, -3.21419433e+00, -6.87279984e+00, -2.54898468e+00,
  6.75719900e+00,  3.36281405e+00,  1.69972374e+00,  5.73625563e+00,
  -6.18932675e+00, -8.24400086e-01,  1.45918085e+01, -4.99246572e+00,
  -4.24917941e+00, -8.34416037e+00,  1.66723485e+00,  6.73274369e+00,
  8.84794082e-01, -6.92253212e+00,  1.08290913e+01, -1.21100351e+00,
  -4.34763202e+00, -9.24909757e+00, -5.85098671e+00,  7.28446041e+00])

assert np.allclose(targets, expected_targets), "Unexpected targets"

expected_predictions = np.array([
  -6.43799132e+00,  4.16380343e-01, -4.62929848e+00, -4.60695748e+00,
  -3.72501996e+00,  1.07307041e+00, -6.19533751e+00,  3.34913326e-01,
  3.08086826e+00, -2.45359210e+00, -4.79420628e+00,  9.01328411e+00,
  1.36905481e+00,  6.94309918e+00,  5.12989112e+00, -5.93387031e+00,
  4.98791299e+00,  8.62502069e-01, -1.45124245e+00, -2.60646328e-01,
  3.55027090e+00,  3.26200932e+00,  1.77278581e+00, -5.93008381e+00,
  -2.07272056e+00, -5.13679365e+00, -2.59582488e+00,  8.52439182e+00,
  -4.03890200e+00,  5.58085427e+00, -1.22244180e+00,  1.69303612e+00,
  2.82816841e+00,  2.33968575e+00,  2.08697185e-01, -1.08757018e+01,
  -1.46024130e+00,  8.60414536e+00,  2.21174975e+00,  5.20732142e+00,
  -5.21543281e+00, -1.72663157e+00,  1.58085417e+00,  4.72270699e+00,
  4.19126804e+00, -2.53284920e+00,  4.78167976e+00, -1.44621047e+00,
  -7.51574146e+00,  1.08925103e+00, -2.01575786e+00,  4.15713318e+00,
  -6.19753275e+00, -6.69887689e+00,  1.93459766e-01,  4.32719268e+00,
  -4.84819661e+00,  3.06008227e+00, -5.14331806e+00, -4.91513863e+00,
  3.09155130e+00, -2.14557559e+00, -1.74243123e+00,  6.91851834e+00,
  5.20994520e+00, -1.33072195e-01,  7.01623808e-01,  8.24763018e+00,
  8.14145573e+00, -8.32893674e+00, -6.56434899e+00, -2.32164594e+00,
  -2.18805332e-01, -4.99087344e+00,  4.06156092e+00, -2.31544485e+00,
  1.17378984e+01,  5.48263571e+00,  8.02570711e+00, -3.52716385e+00,
  -3.23427551e+00, -1.75722815e+00, -2.89565899e+00,  4.06849592e+00,
  -1.63333847e+00,  4.04786431e-01,  7.67944276e-01, -8.33528463e+00,
  6.49269125e+00, -3.82907179e+00, -2.60651889e+00, -2.83563598e+00,
  5.04278745e+00,  3.41910700e+00,  1.52427310e+00,  7.30084839e+00,
  -1.00949339e+01, -5.16921455e+00,  4.49842043e+00, -5.98980638e+00,
  4.31447701e+00, -7.93222438e-01, -6.86706698e+00,  7.44077093e+00,
  6.88468284e+00, -7.02372312e+00, -8.52363739e-01, -2.19982252e+00,
  -9.07752775e-01,  1.60644269e+00,  4.96423112e+00, -2.84342735e+00,
  7.73062651e+00,  1.28760679e+01,  1.03257519e+01, -6.34708904e+00,
  -4.72806598e+00,  3.35136344e+00,  6.88877928e-01, -9.83697660e-01,
  4.10162193e+00,  9.35821908e+00,  1.00809626e+01, -3.04479929e+00,
  -3.94003162e+00, -3.31454557e+00, -4.62724592e+00,  6.23825341e+00,
  -3.93526629e+00, -3.02498635e+00, -1.70807378e+00, -3.18783406e+00,
  -3.49036423e+00, -1.18379270e+01, -3.43880613e+00, -6.24251833e-01,
  -2.82313671e+00,  3.90552130e-03,  2.53875523e+00,  5.32096428e+00,
  -5.97469491e+00, -3.62479194e+00, -2.58049970e+00, -6.77678420e+00,
  -2.59203809e+00, -1.74763976e+00,  8.70547924e+00, -3.89090957e+00,
  1.15639584e+00,  1.48476782e+00, -2.43651286e+00, -9.07107014e+00,
  4.61230449e+00, -3.78406976e+00,  8.60148921e+00,  3.36436279e+00,
  -6.13243231e+00, -1.06315843e+00, -6.85735450e+00, -1.36887485e+00,
  -1.09174866e-01,  6.01532304e+00,  5.69363617e-01,  5.05563121e+00,
  -7.42939940e+00, -6.02993479e+00,  5.33848977e+00, -3.91591889e+00,
  -5.88326498e+00, -9.58014200e-01,  5.64049539e+00,  1.01365472e+00,
  7.93802804e-01, -7.27131142e+00, -7.58196859e-01,  6.09134175e+00,
  -3.58163898e+00, -6.62841460e+00, -6.00627817e+00, -1.50342700e+00,
  2.59742583e+00, -4.64702135e+00, -4.40600144e+00,  1.61626580e+00,
  -4.05204042e+00, -2.19455925e+00,  8.84095100e+00, -1.49774257e+00,
  3.87702030e+00, -4.67585683e+00,  3.79334810e+00,  9.92097624e+00,
  5.64554527e-01,  1.06030938e+00,  1.07518517e+00, -7.61947534e+00,
  2.03588810e+00, -5.26016592e+00, -1.91634902e+00, -2.62365564e+00])

assert np.allclose(predictions, expected_predictions), "Unexpected predictions"

'LGTM'